In [1]:
import re
from pathlib import Path
from os.path import abspath
import os
import numpy as np
import pandas as pd
import random
import pickle
import csv
import subprocess

In [2]:
print(os.getcwd())

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/analysis/pmi_verification


In [3]:
stimuli_path = abspath('../../ressources/scrambled_stimuli_dfs')
print(stimuli_path)

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/ressources/scrambled_stimuli_dfs


In [4]:
fname = "stimuli_Original.pkl"
with open(os.path.join(stimuli_path,fname), 'rb') as f:
    df = pickle.load(f)
df

Loading lookup from /om2/user/ckauf/anaconda/envs/perturbedenv/lib/python3.6/site-packages/brainio_collection/lookup.csv
/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/neural_nlp/../ressources/stimuli


 We're running in the NEW version of the implementations.py script.




/om2/user/ckauf/anaconda/envs/perturbedenv/lib/python3.6/site-packages/brainio_base/assemblies.py:224: FutureWarning: The inplace argument has been deprecated and will be removed in a future version of xarray.
  xr_data.set_index(append=True, inplace=True, **coords_d)


,sentence,sentence_num,stimulus_id,experiment,story,passage_index,passage_label,passage_category
0,beekeeping encourages the conservation of loca...,0,243sentences.0,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
1,it is in every beekeeper's interest to conserv...,1,243sentences.1,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
2,as a passive form of agriculture it does not r...,2,243sentences.2,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
3,beekeepers also discourage the use of pesticid...,3,243sentences.3,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
4,artisanal beekeepers go to extremes for their ...,4,243sentences.4,243sentences,243sentences.beekeeping,2,beekeeping,beekeeping
...,...,...,...,...,...,...,...,...
622,some windows have multiple panes to increase i...,379,384sentences.379,384sentences,384sentences.building_part,95,Window,building_part
623,a woman is a female human adult.,380,384sentences.380,384sentences,384sentences.human,96,Woman,human
624,a woman is stereotypically seen as a caregiver.,381,384sentences.381,384sentences,384sentences.human,96,Woman,human
625,a woman can become pregnant and bear children.,382,384sentences.382,384sentences,384sentences.human,96,Woman,human


In [5]:
# Mollica calculate PMI input style is:
# * 1. column is stimulus_id
# * 2. column is condition
# * 3. column is sentence

In [6]:
conditions = ['Original', 'Scr1', 'Scr3', 'Scr5', 'Scr7', 'lowPMI', 'lowPMI_random', 'backward', 'random', 'random_poscontrolled', 'random_withreplacement']

In [7]:
frames = []

savedir = abspath('./stimuli_csvs')
os.makedirs(savedir, exist_ok=True)

for cond in conditions:
    for filename in os.listdir(stimuli_path):
        if not ".pkl" in filename:
            continue
        if filename.split(".")[0].split("stimuli_")[1] == cond:
            print(cond)
            with open(os.path.join(stimuli_path,filename), 'rb') as f:
                df = pickle.load(f)
            ids = list(df["stimulus_id"])
            conds = list([cond]*len(ids))
            sentences = list(df["sentence"])
            #print(ids[:5], conds[:5], sentences[:5])
            #print("*"*30)
            curr_df = pd.DataFrame({
                "id" : ids,
                "condition" : conds,
                "sentence" : sentences
            })
            
    curr_df.to_csv(f"{savedir}/stimuli_{cond}.csv", header=None, index=False)
    frames.append(curr_df)

Original
Scr1
Scr3
Scr5
Scr7
lowPMI
lowPMI_random
backward
random
random_poscontrolled
random_withreplacement


In [8]:
full_df = pd.concat(frames)
full_df

,id,condition,sentence
0,243sentences.0,Original,beekeeping encourages the conservation of loca...
1,243sentences.1,Original,it is in every beekeeper's interest to conserv...
2,243sentences.2,Original,as a passive form of agriculture it does not r...
3,243sentences.3,Original,beekeepers also discourage the use of pesticid...
4,243sentences.4,Original,artisanal beekeepers go to extremes for their ...
...,...,...,...
622,384sentences.379,random_withreplacement,and the depending allow watching throwing ofte...
623,384sentences.380,random_withreplacement,used things that connected and a to.
624,384sentences.381,random_withreplacement,joined the the drunk the or have due.
625,384sentences.382,random_withreplacement,controlled barrier butterfat on articles their...


In [9]:
print(list(full_df.loc[full_df["condition"] == "random_poscontrolled"]["sentence"])[:5])

['legs emphasize to jacket up throwing large.', 'mountaineers cool on a spent waters small a in gambling surroundings composition went.', 'to in is as backgrounds unlocked streamlined a decoration wood can turn can sand is house spread slammed a internal.', 'have animal television and studies and need one the the sometimes dark cooking can start.', 'a volunteer desk and scrambling down and structures forks making ice is care personal to.']


In [10]:
full_df.to_csv(f"{savedir}/all_scrambled_stimuli.csv", header=None, index=False)